# Basic SQL Queries with Postgres

For this notebook series, I'll be using exercises from <a href='https://www.pgexercises.com'>pgexercises.com</a>, using data from a hypothetical country club with information on facility usage. You can download the dataset <a href='https://www.pgexercises.com/dbfiles/clubdata.sql'>here</a>. The 3 tables in the dataset have the following format:

    CREATE TABLE cd.members
    (
       memid integer NOT NULL, 
       surname character varying(200) NOT NULL, 
       firstname character varying(200) NOT NULL, 
       address character varying(300) NOT NULL, 
       zipcode integer NOT NULL, 
       telephone character varying(20) NOT NULL, 
       recommendedby integer,
       joindate timestamp not null,
       CONSTRAINT members_pk PRIMARY KEY (memid),
       CONSTRAINT fk_members_recommendedby FOREIGN KEY (recommendedby)
       REFERENCES cd.members(memid) ON DELETE SET NULL
    );
    
    CREATE TABLE cd.facilities
    (
       facid integer NOT NULL, 
       name character varying(100) NOT NULL, 
       membercost numeric NOT NULL, 
       guestcost numeric NOT NULL, 
       initialoutlay numeric NOT NULL, 
       monthlymaintenance numeric NOT NULL, 
       CONSTRAINT facilities_pk PRIMARY KEY (facid)
    );
    
    CREATE TABLE cd.bookings
    (
       bookid integer NOT NULL, 
       facid integer NOT NULL, 
       memid integer NOT NULL, 
       starttime timestamp NOT NULL,
       slots integer NOT NULL,
       CONSTRAINT bookings_pk PRIMARY KEY (bookid),
       CONSTRAINT fk_bookings_facid FOREIGN KEY (facid) REFERENCES cd.facilities(facid),
       CONSTRAINT fk_bookings_memid FOREIGN KEY (memid) REFERENCES cd.members(memid)
    );

First, we need to load up the dataset into postgres

In [ ]:
psql - U postgres -f clubdata.sql -d postgres -x -q

I'm going to create a new user for this project, using the command line from the postgres superuser.

In [ ]:
psql -U postgres

then I created a new user james for the project:

In [ ]:
CREATE ROLE james WITH LOGIN PASSWORD 'fakepassword';

granting privileges on our new table:

In [ ]:
GRANT ALL PRIVILEGES ON exercises TO james;

Now it's time to dig in with python's psycopg2 interface.

First I boot up all the db and set my helper functions. I'm displaying them here for reference, but since I resuse them in all the notebooks of the series, I have this written in a separate pgexercises.py file, and load it up into the cell in the first magic. 

In [132]:
# %load pgexercises.py
#import postgres interface for python
import psycopg2
import os
import json
import pandas as pd
from IPython.display import display

#keeping output from each cell a little smaller for easy scrolling.
pd.set_option('display.max_rows',14)

with open('config.json') as f:
    conf = json.load(f) 

postgres_string = "dbname='{database}' user='{user}' host='{host}' password='{passw}'".format(**conf)

relative_data_path = '../../Data/SQL'
database_filename = 'clubdata.sql'

try:
    conn = psycopg2.connect("dbname='exercises' user='james' host='localhost' password='fakepassword'")
except:
    print("couldn't connect")
cursor = conn.cursor()

#helper function for printing. For better jupyter formatting, I'm just using panda's built in read function. 
def print_output(query):
    global conn
    display(pd.read_sql_query(query, conn))

Writing text.py


## Examine one of the tables
We'll start by looking at the facilities table. After this cell I'll be using a little helper function to convert the output into a pandas dataframe and display using jupyter's built in display formatting. In this first cell, I also show how you can get the column names from the cursor description. 

In [112]:
cursor.execute("SELECT * FROM cd.facilities;")

#how to get the column names 
colnames = [desc[0] for desc in cursor.description]
print('column names are:\n',colnames,"\n")

rows = cursor.fetchall()
for row in rows:
    print(row)

column names are:
 ['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance'] 

(0, 'Tennis Court 1', Decimal('5'), Decimal('25'), Decimal('10000'), Decimal('200'))
(1, 'Tennis Court 2', Decimal('5'), Decimal('25'), Decimal('8000'), Decimal('200'))
(2, 'Badminton Court', Decimal('0'), Decimal('15.5'), Decimal('4000'), Decimal('50'))
(3, 'Table Tennis', Decimal('0'), Decimal('5'), Decimal('320'), Decimal('10'))
(4, 'Massage Room 1', Decimal('35'), Decimal('80'), Decimal('4000'), Decimal('3000'))
(5, 'Massage Room 2', Decimal('35'), Decimal('80'), Decimal('4000'), Decimal('3000'))
(6, 'Squash Court', Decimal('3.5'), Decimal('17.5'), Decimal('5000'), Decimal('80'))
(7, 'Snooker Table', Decimal('0'), Decimal('5'), Decimal('450'), Decimal('15'))
(8, 'Pool Table', Decimal('0'), Decimal('5'), Decimal('400'), Decimal('15'))


To get just the name and cost columns:

In [113]:
print_output("SELECT name,membercost FROM cd.facilities;")

,name,membercost
0,Tennis Court 1,5.0
1,Tennis Court 2,5.0
2,Badminton Court,0.0
3,Table Tennis,0.0
4,Massage Room 1,35.0
5,Massage Room 2,35.0
6,Squash Court,3.5
7,Snooker Table,0.0
8,Pool Table,0.0


See all the information on facilities that charge an additional fee to users for use:

In [108]:
print_output("SELECT * FROM cd.facilities WHERE membercost > 0;")

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000.0,200.0
1,1,Tennis Court 2,5.0,25.0,8000.0,200.0
2,4,Massage Room 1,35.0,80.0,4000.0,3000.0
3,5,Massage Room 2,35.0,80.0,4000.0,3000.0
4,6,Squash Court,3.5,17.5,5000.0,80.0


Get the 4 relevant columns for the rows that charge a member cost, but that cost is under 1/50th the monthly maintenance cost of that facility. 

In [114]:
q = """
    SELECT facid,name,membercost,monthlymaintenance 
    FROM cd.facilities 
    WHERE membercost > 0 AND membercost < monthlymaintenance/50;"""

print_output(q)

,facid,name,membercost,monthlymaintenance
0,4,Massage Room 1,35.0,3000.0
1,5,Massage Room 2,35.0,3000.0


Get all the rows where the 'name' column containts the substring 'Tennis'

In [115]:
print_output("SELECT * FROM cd.facilities WHERE name LIKE '%Tennis%';")

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000.0,200.0
1,1,Tennis Court 2,5.0,25.0,8000.0,200.0
2,3,Table Tennis,0.0,5.0,320.0,10.0


See all the rows where facid is equal to 1 or 5

In [116]:
print_output("SELECT * FROM cd.facilities WHERE facid IN (1,5);")

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000.0,200.0
1,5,Massage Room 2,35.0,80.0,4000.0,3000.0


How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive' depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.

In [117]:
q = """SELECT name, 
        CASE WHEN monthlymaintenance > 100 THEN 'expensive' 
            ELSE 'cheap' END AS cost
        FROM cd.facilities;"""

print_output(q)

,name,cost
0,Tennis Court 1,expensive
1,Tennis Court 2,expensive
2,Badminton Court,cheap
3,Table Tennis,cheap
4,Massage Room 1,expensive
5,Massage Room 2,expensive
6,Squash Court,cheap
7,Snooker Table,cheap
8,Pool Table,cheap


How can you produce a list of members who joined after the start of September 2012? Return the memid, surname, firstname, and joindate of the members in question.

In [118]:
print_output("SELECT memid, surname, firstname, joindate FROM cd.members WHERE joindate > '2012-08-31';")

,memid,surname,firstname,joindate
0,24,Sarwin,Ramnaresh,2012-09-01 08:44:42
1,26,Jones,Douglas,2012-09-02 18:43:05
2,27,Rumney,Henrietta,2012-09-05 08:42:35
3,28,Farrell,David,2012-09-15 08:22:05
4,29,Worthington-Smyth,Henry,2012-09-17 12:27:15
5,30,Purview,Millicent,2012-09-18 19:04:01
6,33,Tupperware,Hyacinth,2012-09-18 19:32:05
7,35,Hunt,John,2012-09-19 11:32:45
8,36,Crumpet,Erica,2012-09-22 08:36:38
9,37,Smith,Darren,2012-09-26 18:08:45


How can you produce an ordered list of the first 10 surnames in the members table? The list must not contain duplicates.

In [119]:
print_output("SELECT DISTINCT surname FROM cd.members ORDER BY surname LIMIT 10;")

,surname
0,Bader
1,Baker
2,Boothe
3,Butters
4,Coplin
5,Crumpet
6,Dare
7,Farrell
8,Genting
9,GUEST


You, for some reason, want a combined list of all surnames and all facility names. Yes, this is a contrived example. Produce that list!

In [124]:
print_output("SELECT surname FROM cd.members UNION SELECT name FROM cd.facilities;")

,surname
0,Tennis Court 2
1,Worthington-Smyth
2,Badminton Court
3,Pinker
4,Dare
5,Bader
6,Mackenzie
...,...
27,Farrell
28,Tennis Court 1


You'd like to get the signup date of your last member. How can you retrieve this information?

In [127]:
print_output("SELECT max(joindate) AS most_recent_join_date FROM cd.members;")

,most_recent_join_date
0,2012-09-26 18:08:45


In [128]:
print_output("""SELECT firstname, surname, joindate FROM cd.members WHERE joindate = 
(SELECT max(joindate) FROM cd.members);""")

,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45
